# Process Habermann et al.

## Set up Env

In [ ]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

In [ ]:
import liana as li

In [ ]:
from prep_utils import filter_samples, filter_celltypes, map_gene_symbols, check_group_balance

In [ ]:
dataset = 'habermann'
groupby = 'celltype'
sample_key = 'Sample_Name'
condition_key = 'Status'
batch_key = 'Sample_Source'

min_cells_per_sample = 700
sample_zcounts_max = 3
sample_zcounts_min = -2

# set filtering parameters
min_cells = 20 # min number of cells per cell type
min_samples = 5 # min number of samples that pass the threshold per cell type

In [1]:
from processer import DatasetHandler

In [2]:
processer = DatasetHandler(dataset_name = 'habermann')

In [3]:
adata = processer.process_dataset()

         Sample_Name
Status              
Control            9
ILD               17
Groups are imbalanced!
Status
Control     9
ILD        17
Name: Status, dtype: int64
Balancing groups...


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/contextlib.py:142: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
Now running: VUILD65: 100%|██████████| 18/18 [09:25<00:00, 31.44s/it]


## Preprocess

### Load data

In [ ]:
adata = sc.read_h5ad(os.path.join('data', f"{dataset}.h5ad"), backed='r')
adata

In [ ]:
adata.obs[[sample_key, condition_key]].drop_duplicates().nunique()

#### Filter samples acc to QC

In [ ]:
adata = filter_samples(adata, sample_key, condition_key, min_cells_per_sample, sample_zcounts_max, sample_zcounts_min)

### Check Group balance

In [ ]:
if adata.is_view:
    adata = adata.to_memory()

In [ ]:
adata = check_group_balance(adata, condition_key, sample_key)

In [ ]:
adata.obs[[condition_key, sample_key]].drop_duplicates().groupby(adata.obs[condition_key]).count()

Filter cell types, and genes

In [ ]:
adata = filter_celltypes(adata=adata, groupby=groupby, sample_key=sample_key, min_cells=min_cells, min_samples=min_samples)

In [ ]:
# Remove genes expressed in few cells
sc.pp.filter_genes(adata, min_cells=min_cells)
adata

### Normalize

In [ ]:
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

### Add Metadata

In [ ]:
adata.uns['sample_key'] = sample_key
adata.uns['batch_key'] = batch_key
adata.uns['condition_key'] = condition_key
adata.uns['groupby'] = groupby


In [ ]:
assert np.isin(['sample_key', 'batch_key', 'condition_key', 'groupby'], adata.uns_keys()).all()

## Run LIANA

In [ ]:
import liana as li

In [ ]:
li.mt.rank_aggregate.by_sample(adata, groupby=groupby, use_raw=False, sample_key=sample_key, verbose=True, n_perms=None)

### Write

In [ ]:
adata.write_h5ad(os.path.join('data', 'interim', f"{dataset}_processed.h5ad"))

## Run Dimensionality Reductions

In [ ]:
import os
import scanpy as sc
from classify_utils import dim_reduction_pipe

In [ ]:
adata = sc.read_h5ad(os.path.join('data', 'interim', f"{dataset}_processed.h5ad"), backed='r')

In [ ]:
dim_reduction_pipe(adata, dataset)

## Run Classifier

In [ ]:
import os
import scanpy as sc
from classify_utils import run_classifier

In [ ]:
adata = sc.read_h5ad(os.path.join('data', 'results', f"{dataset}_dimred.h5ad"), backed='r')

In [ ]:
run_classifier(adata, dataset)

In [ ]:
adata.uns['evaluate']

In [ ]:
adata.uns['auc'].sort_values(by=['score_key', 'reduction_name', 'fold'])